Mudanças no Consumo por Região ao Longo do Tempo

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import regexp_replace, col

spark = SparkSession.builder \
    .appName("ConsumoPorRegiao") \
    .getOrCreate()

In [7]:
df_vendas_atual = spark.read.parquet("../../datalake/silver/Liquidos_Vendas_Atual")

In [8]:
df_vendas_atual = df_vendas_atual.withColumn(
    "quantidade_de_produto_(mil_m3)",
    regexp_replace("quantidade_de_produto_(mil_m3)", ",", ".").cast("double")
)

In [9]:
ranking_estados = df_vendas_atual.groupBy("uf_destino") \
    .agg(sum("quantidade_de_produto_(mil_m3)").alias("volume_total_(mil_m3)")) \
    .orderBy(col("volume_total_(mil_m3)").desc())

ranking_estados.show()

+----------+---------------------+
|uf_destino|volume_total_(mil_m3)|
+----------+---------------------+
|        SP|    259378.8472289994|
|        MG|   115110.90283499996|
|        PR|    85035.09810300011|
|        RS|    62012.21745100026|
|        BA|     50848.5593370005|
|        GO|    50027.79802700086|
|        SC|   48638.686370000505|
|        RJ|   44269.873204000294|
|        MT|    40423.03944900033|
|        PA|    37357.05099100023|
|        PE|   27226.471624000285|
|        MA|   24921.451695000138|
|        CE|   22009.600614000206|
|        MS|   21575.763193000057|
|        ES|   19169.090615000125|
|        AM|   14614.327326000044|
|        DF|    13078.76899900003|
|        TO|   12926.388347000038|
|        PB|   11827.744895000045|
|        RO|   11548.004939999999|
+----------+---------------------+
only showing top 20 rows


In [10]:
polos_distribuicao_consumo = df_vendas_atual.groupBy("uf_origem", "uf_destino") \
    .agg(sum("quantidade_de_produto_(mil_m3)").alias("volume_total_(mil_m3)")) \
    .orderBy(col("volume_total_(mil_m3)").desc())

polos_distribuicao_consumo.show()

+---------+----------+---------------------+
|uf_origem|uf_destino|volume_total_(mil_m3)|
+---------+----------+---------------------+
|       SP|        SP|    257832.4173899994|
|       MG|        MG|    94367.65813800006|
|       PR|        PR|     84335.3453140001|
|       RS|        RS|   61829.702941000185|
|       BA|        BA|    49918.17240800019|
|       GO|        GO|   45721.698671000384|
|       RJ|        RJ|   43735.663893000215|
|       SC|        SC|    40706.72650600013|
|       MT|        MT|    34394.85501400011|
|       PA|        PA|    34323.57635900008|
|       PE|        PE|   25988.348110000108|
|       MA|        MA|    24615.86661000007|
|       CE|        CE|    18957.95149400005|
|       ES|        ES|    17888.63450400003|
|       MS|        MS|    16549.56698400001|
|       AM|        AM|   13945.034194000005|
|       DF|        DF|   12721.474400000021|
|       SP|        MG|   11756.270066000014|
|       RO|        RO|         11483.211751|
|       PB

In [12]:
consumo_por_ano_estado = df_vendas_atual.groupBy("ano", "uf_destino") \
    .agg(sum("quantidade_de_produto_(mil_m3)").alias("volume_total_(mil_m3)")) \
    .orderBy("ano", col("volume_total_(mil_m3)").desc())

consumo_por_ano_estado.show()

+----+----------+---------------------+
| ano|uf_destino|volume_total_(mil_m3)|
+----+----------+---------------------+
|2017|        SP|   30245.838844000045|
|2017|        MG|   12989.312391000016|
|2017|        PR|    9583.654475999996|
|2017|        RS|    7230.573206999993|
|2017|        BA|    5869.082703999992|
|2017|        GO|    5407.690431999988|
|2017|        RJ|    5403.513036999995|
|2017|        SC|   5366.1512689999945|
|2017|        PA|    4258.196008999997|
|2017|        MT|   4005.4697439999977|
|2017|        PE|    3416.910594999999|
|2017|        MA|    2770.258344999998|
|2017|        CE|   2509.1126599999975|
|2017|        MS|   2118.6339609999995|
|2017|        ES|   2092.1087399999988|
|2017|        AM|   1755.5556749999992|
|2017|        DF|   1632.7742669999996|
|2017|        PB|   1533.6921139999993|
|2017|        TO|   1318.1686889999996|
|2017|        RO|   1277.5953570000001|
+----+----------+---------------------+
only showing top 20 rows


In [13]:
ranking_estados.write.mode("overwrite").parquet("../../datalake/gold/ranking_estados")
polos_distribuicao_consumo.write.mode("overwrite").parquet("../../datalake/gold/polos_distribuicao_consumo")
consumo_por_ano_estado.write.mode("overwrite").parquet("../../datalake/gold/consumo_por_ano_estado")